In [1]:
import os
os.chdir("data")
os.getcwd()

'/Users/williamlee/Documents/Lego Sorter/Annotation Image Cropper/PascalVOC-to-Images/data'

In [30]:
import glob
import os
import xmltodict
import json
import pprint

from PIL import Image
import cv2
from matplotlib import pyplot as plt

# Printing results
pp = pprint.PrettyPrinter(indent=4)


# Look for XML files and parses then as if they were Pascal VOC Files
def process():
    # Finds all XML files on data/ and append to list
    pascal_voc_contents = []
    #os.chdir("data")

    print("Found {} files in data directory!".format(
        str(len(glob.glob("*.xml")))))
    for file in glob.glob("*.xml"):
        f_handle = open(file, 'r')
        print("Parsing file '{}'...".format(file))
        pascal_voc_contents.append(xmltodict.parse(f_handle.read()))
        

    # Process each file individually
    for index in pascal_voc_contents:
        image_file = index['annotation']['filename']
        print(image_file)
        print("Image file '{}' printed here ".format(image_file))
        #image_file = image_file + '.jpg'
        image_file = image_file 
        # If there's a corresponding file in the folder,
        # process the images and save to output folder
        if os.path.isfile(image_file):
            dataset = index['annotation']
            if 'object' in dataset:
                extractDataset(dataset)
               # break
        else:
            print("Image file '{}' not found, skipping file...".format(image_file))


# Extract image samples and save to output dir
def extractDataset(dataset):
    if 'object' in dataset:
        print("Found {} objects on image '{}'...".format(
            len(dataset['object']), dataset['filename']))

        img = Image.open(dataset['filename'] )
        i = 0

        # Run through each item and save cut image to output folder
        if isinstance(dataset['object'], list):
            for item in dataset['object']:
                bndbox = dict([( a, float(b.replace(',','.'))) for (a, b) in item['bndbox'].items()])
                
                # Create output directory
                save_dir = item['name'] 
                try:
                    os.mkdir(save_dir)
                except:
                    pass
                ySum = 1280-1061
                
                im = img.crop((bndbox['xmin'], bndbox['ymin'],
                               bndbox['xmax'], bndbox['ymax']))
                
                
                #IMAGE POSTPROCESSING HERE (grayscale, pixelate, )
                
                #Convert gray scale
                im = im.convert('L')
                
                #Pixelate image by factor defined by pix_factor
                pix_factor = 3
                im = im.resize((int(im.width/pix_factor), int(im.height/pix_factor)))
                im = im.resize((im.width*pix_factor, im.height*pix_factor), Image.NEAREST)
                
                
                im.save( save_dir + "/" + dataset['filename'] + str(i) + '.jpg')
                i = i + 1
        else:
            print('the else is here')
            bndbox = dict([( a, float(b.replace(',','.'))) for (a, b) in dataset['object']['bndbox'].items()])
                
            print(dataset['object']['name'])
            # Create output directory
            save_dir = dataset['object']['name']
            try:
                os.mkdir(save_dir)
            except:
                pass
            ySum = 1280-1061
            im = img.crop((bndbox['xmin'], bndbox['ymin'] ,
                           bndbox['xmax'], bndbox['ymax'] ))
            
            # Save
            #im.save(sample_preamble + str(i) + '.jpg')
            im.save( save_dir + "/" + dataset['filename'] + str(i) + '.jpg')
            #i = i + 1


process()


Found 1 files in data directory!
Parsing file 'test1.xml'...
test1.jpg
Image file 'test1.jpg' printed here 
Found 4 objects on image 'test1.jpg'...
(56, 92)
(184, 96)
(116, 116)
(92, 164)
